<a href="https://colab.research.google.com/github/Alinananaaa/2025_programming/blob/main/W4_0313_%E9%9A%A8%E5%A0%82%E4%BB%BB%E5%8B%99%E7%B9%B3%E4%BA%A4_OOXX%E9%81%8A%E6%88%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe
import pandas as pd

In [23]:

# Set up Google Gemini AI
apikey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

# Initialize the Tic-Tac-Toe board
def initialize_board():
    return [[' ' for _ in range(3)] for _ in range(3)]

# Display the board
def print_board(board):
    for row in board:
        print(" | ".join(row))
        print("-" * 9)

# Get available moves
def get_available_moves(board):
    return [(r, c) for r in range(3) for c in range(3) if board[r][c] == ' ']

# AI makes a random move
def ai_make_move(board, symbol):
    available_moves = get_available_moves(board)
    if available_moves:
        move = random.choice(available_moves)
        board[move[0]][move[1]] = symbol
        return move
    return None

# Check for a winner
def check_winner(board):
    for row in board:
        if row[0] == row[1] == row[2] != ' ':
            return row[0]

    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != ' ':
            return board[0][col]

    if board[0][0] == board[1][1] == board[2][2] != ' ':
        return board[0][0]

    if board[0][2] == board[1][1] == board[2][0] != ' ':
        return board[0][2]

    return None

# AI chat function
def ai_chat(player, move, board):
    prompt = f"AI {player} made a move at {move}. Current board:\n"
    for row in board:
        prompt += " | ".join(row) + "\n"
    response = model.generate_content(prompt)
    return response.text if response else "(AI had no response)"

# Play a round
def play_tic_tac_toe():
    board = initialize_board()
    players = ['X', 'O']
    conversation_log = []
    for turn in range(9):
        current_player = players[turn % 2]
        move = ai_make_move(board, current_player)
        chat_response = ai_chat(current_player, move, board)
        conversation_log.append([f"AI {current_player}", str(move), chat_response])
        print_board(board)

        winner = check_winner(board)
        if winner:
            conversation_log.append(["Game Over", "Winner:", f"AI {winner}"])
            break

    if not winner:
        conversation_log.append(["Game Over", "Result:", "Draw"])

    return conversation_log


In [24]:
# Authenticate with Google Sheets
def authenticate_google_sheets():
    creds, _ = default()
    return gspread.authorize(creds)

In [25]:
# Export to Google Sheets
def export_to_google_sheets(data, sheet_url):
    gc = authenticate_google_sheets()
    sheet = gc.open_by_url(sheet_url).sheet1
    df = pd.DataFrame(data, columns=['Player', 'Move', 'Chat'])
    set_with_dataframe(sheet, df)
    print("✅ Successfully exported to Google Sheets!")

In [26]:
# Run the game twice and export
if __name__ == "__main__":
    auth.authenticate_user()  # Ensure manual authentication in Google Colab
    google_sheets_url = "https://docs.google.com/spreadsheets/d/1VUjumBwc_wUdoAwEzYk3tSa0SQRDNlQ-WQiRQ1gcGZA/edit?usp=sharing"

    full_conversation = []
    for round_no in range(2):
        print(f"Round {round_no + 1}")
        round_data = play_tic_tac_toe()
        full_conversation.extend(round_data)
        time.sleep(2)

    # Export to Google Sheets
    export_to_google_sheets(full_conversation, google_sheets_url)

Round 1
  |   |  
---------
  |   | X
---------
  |   |  
---------
  |   |  
---------
  | O | X
---------
  |   |  
---------
  |   | X
---------
  | O | X
---------
  |   |  
---------
  |   | X
---------
  | O | X
---------
  | O |  
---------
  |   | X
---------
  | O | X
---------
  | O | X
---------
Round 2
  | X |  
---------
  |   |  
---------
  |   |  
---------
O | X |  
---------
  |   |  
---------
  |   |  
---------
O | X |  
---------
  |   |  
---------
X |   |  
---------
O | X |  
---------
  |   | O
---------
X |   |  
---------
O | X |  
---------
  |   | O
---------
X |   | X
---------
O | X |  
---------
  | O | O
---------
X |   | X
---------
O | X |  
---------
  | O | O
---------
X | X | X
---------
✅ Successfully exported to Google Sheets!
